In [ ]:
#%tensorflow_version 2.5.0

In [ ]:
import os

from google.colab import drive

import cv2
import skimage.io as io
import skimage.transform as trans

import numpy as np 
from numpy import expand_dims
import matplotlib.pyplot as plt
import random

from PIL import Image


from tensorflow import keras

from tensorflow.keras.utils import *
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers

import tifffile as tiff


from tensorflow.python.client import device_lib

#import tensorflow.keras.backend as K
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10392281426812461352
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16183459840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14543393316521343394
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


##Describe the generator

In [ ]:
class DataGen(Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths
        self.n_batches = int(len(self.input_img_paths)/self.batch_size)

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")


        for j, path in enumerate(batch_input_img_paths):

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE).reshape(self.img_size + (1,))
            x[j] = img
 
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE).reshape(self.img_size+(1,))
            y[j] = img
            
        return x/256, y/256

##Fetch data

In [ ]:
drive.mount('/content/drive')

dataset = "ISBI2016"

path = "/content/drive/MyDrive/"+dataset+"/trainx/"
path_aug = "/content/drive/MyDrive/"+dataset+"/aug_trainx/"
path_y = "/content/drive/MyDrive/"+dataset+"/trainy/"
path_augy = "/content/drive/MyDrive/"+dataset+"/aug_trainy/"

augmented = False

input_img_paths = [path + name for name in os.listdir(path)] ##+ ([path_aug + name for name in os.listdir(path_aug)] if augmented else [])
target_img_paths = [path_y + name for name in os.listdir(path_y)] ##+ ([path_augy + name for name in os.listdir(path_augy)] if augmented else [])
input_img_paths.sort()
target_img_paths.sort()
print(input_img_paths)
print(target_img_paths)
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)

#Get the img shape
img_size = cv2.imread(input_img_paths[0], cv2.IMREAD_GRAYSCALE).shape
print(img_size)

##Describe architecture

In [ ]:
#input size, first layer of the architecture: 572x572
def make_model(input_size, use_batch_normalization=False):
  # first empty layer
  input = layers.Input(input_size)
  #The contracting path (left side of the architecture) follows the typical architecture of a convolutional network.
  # It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), each followed by a rectified linear unit (ReLU)
  # and a 2x2 max pooling operation with stride 2 for downsampling
  
  cnn1 = layers.Conv2D(64,3, activation='relu', padding='same')(input)
  if use_batch_normalization:
    cnn1 = layers.BatchNormalization()(cnn1)
  cnn1 = layers.Conv2D(64,3, activation='relu',padding='same')(cnn1)
  pool1 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(cnn1)
  cnn2 = layers.Conv2D(128,3, activation='relu',padding='same')(pool1)
  if use_batch_normalization:
    cnn2 = layers.BatchNormalization()(cnn2)
  cnn2 = layers.Conv2D(128,3, activation='relu',padding='same')(cnn2)
  pool2 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(cnn2)
  cnn3 = layers.Conv2D(256,3, activation='relu',padding='same')(pool2)
  if use_batch_normalization:
    cnn3 = layers.BatchNormalization()(cnn3)
  cnn3 = layers.Conv2D(256,3, activation='relu', padding='same')(cnn3)
  pool3 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(cnn3)
  cnn4 = layers.Conv2D(512,3, activation='relu', padding='same')(pool3)
  if use_batch_normalization:
    cnn4 = layers.BatchNormalization()(cnn4)
  cnn4 = layers.Conv2D(512,3, activation='relu', padding='same')(cnn4)
  pool4 = layers.MaxPooling2D(pool_size=(2, 2), strides=2)(cnn4)
  cnn5 = layers.Conv2D(1024,3, activation='relu', padding='same')(pool4)
  if use_batch_normalization:
    cnn5 = layers.BatchNormalization()(cnn5)
  cnn5 = layers.Conv2D(1024,3, activation='relu', padding='same')(cnn5)


  #Drop-out layers at the end of the contracting path perform further implicit data augmentation.
  drop_cnn5 = layers.Dropout(0.5)(cnn5)

  #Every step in the expansive path consists of an upsampling of the feature map FOLLOWED (so conv*unsamp) by a 2x2 convolution (“up-convolution”) 
  #that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, 
  #and two 3x3 convolutions, each fol- lowed by a ReLU.

  # P.S. In order to avoid to crop, because I was not sure on how to do it, we just need to put padding='same' 
  cnn_up1 = layers.Conv2D(512,2, activation='relu', padding='same')(layers.UpSampling2D(size = (2,2))(cnn5))
  conc1 = layers.Concatenate(axis=3)([cnn4,cnn_up1])
  cnn6 = layers.Conv2D(512, 3, activation = 'relu', padding='same')(conc1)
  if use_batch_normalization:
    cnn6 = layers.BatchNormalization()(cnn6)
  cnn6 = layers.Conv2D(512, 3, activation = 'relu', padding='same')(cnn6)

  cnn_up2 = layers.Conv2D(256,2, activation='relu', padding='same')(layers.UpSampling2D(size = (2,2))(cnn6))
  conc2 = layers.Concatenate(axis=3)([cnn3,cnn_up2])
  cnn7 = layers.Conv2D(256, 3, activation = 'relu', padding='same')(conc2)
  if use_batch_normalization:
    cnn7 = layers.BatchNormalization()(cnn7)
  cnn7 = layers.Conv2D(256, 3, activation = 'relu', padding='same')(cnn7)

  cnn_up3 = layers.Conv2D(128,2, activation='relu', padding='same')(layers.UpSampling2D(size = (2,2))(cnn7))
  conc3 = layers.Concatenate(axis=3)([cnn2,cnn_up3])
  cnn8 = layers.Conv2D(128, 3, activation = 'relu', padding='same')(conc3)
  if use_batch_normalization:
    cnn8 = layers.BatchNormalization()(cnn8)
  cnn8 = layers.Conv2D(128, 3, activation = 'relu', padding='same')(cnn8)

  cnn_up4 = layers.Conv2D(64,2, activation='relu', padding='same')(layers.UpSampling2D(size = (2,2))(cnn8))
  conc4 = layers.Concatenate(axis=3)([cnn1,cnn_up4])
  cnn9 = layers.Conv2D(64, 3, activation = 'relu', padding='same')(conc4)
  if use_batch_normalization:
    cnn9 = layers.BatchNormalization()(cnn9)
  cnn9 = layers.Conv2D(64, 3, activation = 'relu', padding='same')(cnn9)
  conv9 = layers.Conv2D(2, 3, activation = 'relu', padding = 'same')(cnn9)
  cnn10 = layers.Conv2D(1, 1, activation = 'sigmoid')(cnn9)

  model = keras.Model(inputs = input, outputs = cnn10)

# I am using the base class for keras optimazers, see for more https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
  model.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

##Split data and create model

In [ ]:
#Split data
val_samples = int(len(target_img_paths)*0.7)
test_samples = int(len(target_img_paths)*0.85)

train_input_img_paths = input_img_paths[:val_samples]
train_target_img_paths = target_img_paths[:val_samples]
val_input_img_paths = input_img_paths[val_samples:test_samples]
val_target_img_paths = target_img_paths[val_samples:test_samples]
test_input_img_paths = input_img_paths[test_samples:]
test_target_img_paths = target_img_paths[test_samples:]

print(len(train_input_img_paths),"train,",len(val_input_img_paths),"validation,",len(test_input_img_paths),"test")

batch_size = 4

# Instantiate data Sequences for each split
train_gen = DataGen(batch_size, img_size, train_input_img_paths, train_target_img_paths)
val_gen = DataGen(1, img_size, val_input_img_paths, val_target_img_paths)
test_gen = DataGen(1, img_size, test_input_img_paths, test_target_img_paths)

# Create a model
model = make_model(img_size+(1,), use_batch_normalization=False)

630 train, 135 validation, 135 test


##Functions for measuring performances

In [ ]:
measuring_loss = keras.losses.BinaryCrossentropy(from_logits=False)

def output_image(data, model, title='', save_path=''):
  (X, Y) = data
  X = X[0:1] #Take just one img from the batch
  Y = Y[0:1]
  model_output = model.predict(X)
  f, axarr = plt.subplots(1,3, figsize=(8, 6))
  axarr[0].imshow(X.reshape(img_size))
  axarr[0].title.set_text('Input '+title)
  axarr[1].imshow(model_output.reshape(img_size))
  axarr[1].title.set_text('Output (loss='+str(round(measuring_loss(Y, model_output).numpy(), 5))+')')
  axarr[2].imshow(Y.reshape(img_size))
  axarr[2].title.set_text('Target output')

  if save_path != '':
    f.savefig(save_path, dpi=100, bbox_inches='tight')
  plt.close(f)

def measure_avg_loss(generator, model, max_images_mesures=20):
  avg_loss = 0
  for i in range(generator.n_batches):
    if i * generator.batch_size > max_images_mesures: #Mesurer sur maxi 20 images
      break
    (X, Y) = generator[i]
    model_output = model.predict(X)
    avg_loss += measuring_loss(Y, model_output).numpy() / generator.n_batches
  return avg_loss

def plot_training(epochs, losses, val_losses, save_path):
  plt.figure(figsize=(6, 4))
  # summarize history for loss
  plt.plot(epochs,losses)
  plt.plot(epochs,val_losses)
  plt.title('Training curves')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  if save_path != '':
    plt.savefig(save_path, dpi=100, bbox_inches='tight')
  plt.close()
  

##Train model

In [ ]:
if False:
  save_model_path = "/content/drive/MyDrive/saved_models/augmented/model-{epoch:03d}"

  callbacks = [
    keras.callbacks.EarlyStopping(monitor='loss', patience=3),
    keras.callbacks.ModelCheckpoint(filepath= save_model_path,save_weights_only=True, verbose=0, save_freqs='epochs')
  ]
  # Train the model, doing validation at the end of each epoch.
  epochs = 3

  history = model.fit(train_gen, epochs=epochs, verbose=1, shuffle=True, validation_data=val_gen, callbacks=callbacks)

##Load already trained model

In [ ]:
if False:
  save_model_path = "/content/drive/MyDrive/saved_models/augmented/model-01"
  model = make_model(img_size+(1,), use_batch_normalization=False)
  model.load_weights(save_model_path)

##Measure performance

In [ ]:
model = make_model(img_size+(1,), use_batch_normalization=True)
save_path = "/content/drive/MyDrive/saved_models/with_batch_norm_2016/"
losses = []
val_losses = []
#"weights/model-{epoch:03d}"
history = None

epoch_jump = 1
epochs = [epoch_jump*i for i in range(20)]

for epoch in epochs:
  if history != None:
    losses += history.history['loss']
    val_losses += history.history['val_loss']
    plot_training(epochs[:epoch], losses,val_losses, save_path=save_path+'epoch_'+str(epoch)+'_training')

  output_image(train_gen[0], model, title='(train set)', save_path=save_path+'epoch_'+str(epoch)+'_train')
  output_image(val_gen[0], model, title='(validation set)', save_path=save_path+'epoch_'+str(epoch)+'_val')
  output_image(test_gen[0], model, title='(test set)', save_path=save_path+'epoch_'+str(epoch)+'_test')

  train_loss = round(measure_avg_loss(train_gen, model), 5)
  valid_loss = round(measure_avg_loss(val_gen, model), 5)
  test_loss = round(measure_avg_loss(test_gen, model), 5)
  
  file1 = open(save_path+'epoch_'+str(epoch)+'_losses.txt',"w")#append mode
  file1.write(str(train_loss)+' '+str(valid_loss)+' '+str(test_loss))
  file1.close()

  checkpoint = keras.callbacks.ModelCheckpoint(filepath=save_path+'/weights/epoch_'+str(epoch),save_weights_only=True, verbose=0, save_freqs='epochs')
  history = model.fit(train_gen, epochs=epoch_jump, verbose=1, shuffle=True, validation_data=val_gen, callbacks=[checkpoint])


157/157 [==============================] - 117s 745ms/step - loss: 0.0705 - accuracy: 0.7040 - val_loss: 0.2534 - val_accuracy: 0.6920
